# Populate A New Running Playlist

Here, we will use the insights gathered from my current running playlist to select songs for a new running playlist from from a SQL database of Spotify data. This database is very large and not included with this project, but it can be built from this project [here](https://github.com/tommcd09/spotify-database). This process will likely not produce a perfectly tailored playlist, but the goal is to at least produce a list of songs that I can draw upon to add more songs to my usual rotation.

In [1]:
import sqlite3
import pandas as pd
import config
from spotifyfuncs import authenticate_user, get_playlist_id, populate_playlist

pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## I. Authenticate

Next, we need to authenticate with the Spotify API. Because we will be creating a playlist in my own account, we need to sign in to my account using the prompt_for_user_token method in the spotipy.utils section of the spotipy library [1-2]. From a separate config file, we will pass my username, client id, and client secret, as well as the scope playlist-modify-public, since we will be creating and modifying a public playlist. We will also pass the redirect URI that I listed when registering my project with Spotify.

In [2]:
SCOPE = 'playlist-modify-public'
sp = authenticate_user(config.USERNAME, SCOPE, config.CLIENT_ID, 
                       config.CLIENT_SECRET, 'http://localhost:5000/callback/')

## II. Create New Playlist

Now that we have authenticated, let's create a new playlist called "My New Running Playlist". For now, this will just be an empty playlist [2].

In [3]:
playlist = 'My New Running Playlist'
sp.user_playlist_create(config.USERNAME, name=playlist)

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6RnGdxwzgVyVQ4BwVm4nX3'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/6RnGdxwzgVyVQ4BwVm4nX3',
 'id': '6RnGdxwzgVyVQ4BwVm4nX3',
 'images': [],
 'name': 'My New Running Playlist',
 'owner': {'display_name': 'tommcd09',
  'external_urls': {'spotify': 'https://open.spotify.com/user/tommcd09'},
  'href': 'https://api.spotify.com/v1/users/tommcd09',
  'id': 'tommcd09',
  'type': 'user',
  'uri': 'spotify:user:tommcd09'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSxiNGNmNTc1ZGFjNTExYTU3ZDY1YzlhOTk5YzdjNWZkMmQ3MDBjZjU3',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/6RnGdxwzgVyVQ4BwVm4nX3/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:6RnGdxwzgVyVQ4BwVm4nX3'}

We now have a blank playlist ready to fill.

![Blank Playlist](./images/create_playlist.png)

## III. Select Tracks for New Playlist

Before we can fill this playlist, we need to query our Spotify database for relevant tracks. We can then use the track IDs for these tracks to populate the new playlist.

First, let's open a connection to the Spotify database and create a cursor.

In [4]:
conn = sqlite3.connect('spotify.db')
c = conn.cursor()

Next, we will need to read the top genres and top artists that we identified into Python. We will use these when we query the database.

In [5]:
top_genres = pd.read_csv('./top_genres.csv', index_col=0)
top_genres = list(top_genres.index) #Genre name is the index
top_genres

['hip hop',
 'rap',
 'east coast hip hop',
 'hardcore hip hop',
 'conscious hip hop',
 'gangster rap',
 'alternative hip hop',
 'pop',
 'southern hip hop',
 'rock',
 'political hip hop',
 'album rock',
 'classic rock',
 'dance pop',
 'hard rock',
 'west coast rap',
 'g funk',
 'philly rap',
 'pop rap',
 'queens hip hop',
 'canadian contemporary r&b',
 'canadian pop',
 'art rock']

In [6]:
top_artists = pd.read_csv('./top_artists.csv', index_col=0)
top_artists = list(top_artists.index) #Artist name is the index
top_artists

['The Roots',
 'The Weeknd',
 'Nas',
 'JAY-Z',
 'Rammstein',
 'Black Star',
 'Sia',
 'Aerosmith',
 '2Pac',
 'Kanye West',
 'Ice Cube',
 'Van Halen',
 'Rihanna',
 'Snoop Dogg']

Now, we need to query our database for songs that match <b>either</b> the genres <b>or</b> the artists in the above lists, <b>and</b> that match the relevant audio features we identified in the exploratory analysis:

* Danceability >= 0.52
* Energy >= 0.61
* Speechiness <= 0.26
* Acousticness <= 0.20
* Valence >= 0.39
* Loudness >= -13.95 and <= -2.08
* Time signature = 4
* Tempo >= 87.5 and <= 92.5 OR >= 175 and <= 185

To do this, we will need to:

* Join the tracks, artists, genres, tracks_artists, and genres_artists tables using inner joins.
* Select the track ID, track name, artists, release year, and popularity, as well as various audio features to ensure we selected the correct tracks. 
    * We should select distinct records because joining the tables like this will produce duplicates.
    * We should also select the artists from the tracks table, not the artists table. This will produce just one list of artists per record and reduce duplicates in the results.
* Use a where clause to limit songs to those having the relevant audio features.
* Order the results in a way that makes sense to us. We could order by artist or release date, but for this we will order by descending popularity.

In [7]:
genres_list = "', '".join(top_genres)
artists_list = "', '".join(top_artists)
select_playlist = f"""SELECT DISTINCT
               t.id as track_id, 
               t.name AS track, 
               t.artists, 
               release_year AS release, 
               t.popularity, 
               tempo, 
               danceability, 
               energy, 
               speechiness, 
               acousticness, 
               valence, 
               loudness, 
               time_signature
            FROM tracks AS t
            INNER JOIN tracks_artists AS ta
            ON ta.track_id = t.id
            INNER JOIN artists AS a
            ON ta.artist_id = a.id
            INNER JOIN genres_artists AS ga
            ON ga.artist_id = a.id
            INNER JOIN genres AS g
            ON g.id = ga.genre_id
            WHERE (g.name IN ('{genres_list}')
            OR a.name IN ('{artists_list}'))
            AND t.danceability >= 0.52
            AND t.energy >= 0.61
            AND t.speechiness <= 0.26
            AND acousticness <= 0.20
            AND valence >= 0.39
            AND loudness BETWEEN -13.95 AND -2.08
            AND t.time_signature = 4
            AND ((t.tempo BETWEEN 175 AND 185)
            OR (t.tempo BETWEEN 87.5 AND 92.5))
            ORDER BY t.popularity DESC;"""
new_playlist = pd.read_sql_query(select_playlist, conn)
conn.close()
new_playlist

,track_id,track,artists,release,popularity,tempo,danceability,energy,speechiness,acousticness,valence,loudness,time_signature
0,086myS9r57YsLbJpU0TgK9,Why'd You Only Call Me When You're High?,['Arctic Monkeys'],2013,86,92.00,0.69,0.63,0.04,0.05,0.80,-6.48,4
1,2wrJq5XKLnmhRXHIAf9xBa,"10,000 Hours (with Justin Bieber)","['Dan + Shay', 'Justin Bieber']",2019,85,89.99,0.65,0.63,0.03,0.15,0.43,-4.64,4
2,4jPy3l0RUwlUI9T5XHBW2m,Mood (feat. iann dior),"['24kGoldn', 'iann dior']",2021,85,91.01,0.70,0.72,0.04,0.17,0.73,-3.67,4
3,0qOnSQQF0yzuPWsXrQ9paz,Stereo Hearts (feat. Adam Levine),"['Gym Class Heroes', 'Adam Levine']",2011,84,89.99,0.65,0.80,0.10,0.03,0.80,-3.29,4
4,6FuGJB290AQMAHTfjOYVaK,"Mood (Remix) feat. Justin Bieber, J Balvin & i...","['24kGoldn', 'Justin Bieber', 'J Balvin', 'ian...",2020,82,90.99,0.72,0.74,0.04,0.15,0.70,-3.51,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,1FMuFfitMoCKUJqdPWrhTt,Gangsta Zone,"['Daddy Yankee', 'Snoop Dogg']",2021,1,184.03,0.66,0.88,0.23,0.03,0.66,-4.74,4
731,1X1Kx3taTItaQrBRzKzH22,Gangsta Zone,"['Daddy Yankee', 'Snoop Dogg']",2021,1,184.03,0.66,0.88,0.23,0.03,0.66,-4.74,4
732,47S9ZdzTa2ilVB2JIhwNai,Gangsta Zone,"['Daddy Yankee', 'Snoop Dogg']",2021,1,184.03,0.66,0.88,0.23,0.03,0.66,-4.74,4
733,3VejLq2bIC8ABuTK4DU8lA,Gangsta Zone,"['Daddy Yankee', 'Snoop Dogg']",2021,1,184.03,0.66,0.88,0.23,0.03,0.66,-4.74,4


This produced 735 tracks, and they generally look like they are within the relevant ranges. Some of the tracks appear to be duplicates, but they have unique track IDs in Spotify. Spotify has multiple versions of some songs, so we will just have to select which versions to keep in the playlist once we are done populating it.

## IV. Populate Playlist

Now we will need to identify the Spotify ID for this playlist. We can do this with a function that iterates through a user's playlists and returns the Spotify ID for the playlist with a particular name [2].

In [8]:
playlist_id = get_playlist_id(config.USERNAME, playlist, sp)

We also need to extract a list of track IDs from our query results and prepend them with "spotify:track:" to create a list of Spotify URIs.

In [9]:
playlist_tracks = list(new_playlist['track_id'])
track_uris = ['spotify:track:' + track for track in playlist_tracks]

Finally, we use a custom function to populate our blank playlist with tracks from the list of Spotify URIs. Spotify only allows populating 100 tracks at a time, so this function utilizes a loop to do this.

In [10]:
populate_playlist(config.USERNAME, playlist_id, track_uris, sp)

We now have a Spotify playlist!

![Finished Playlist](./images/populate_playlist.png)

## V. Conclusion

We produced a large but manageable list of songs matching particular audio features that I like while running. Having spent some time listening to the list, it didn't get everything right, but I have quickly discovered new songs that I will be adding to my regular list. In the future, it would be great to try the same task with machine learning.

## VI. References

1. Watts C. December 17, 2021, 'Extracting Song Data From the Spotify API Using Python'. Towards Data Science. https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50.
2. Hochberger E. March 3, 2020, 'Using Python to Create a Spotify Playlist of the Samples on an Album'. Towards Data Science. https://towardsdatascience.com/using-python-to-create-spotify-playlists-of-the-samples-on-an-album-e3f20187ee5e.